In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [16]:
df = pd.read_csv("/content/Creditcard_data.csv")
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [17]:
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0].sample(len(fraud), random_state=42)

balanced_df = pd.concat([fraud, non_fraud]).sample(frac=1, random_state=42)
balanced_df['Class'].value_counts()


,count
Class,
1,9
0,9


In [18]:
X = balanced_df.drop('Class', axis=1)
y = balanced_df['Class']

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [19]:
def simple_random_sampling(X, y):
    return train_test_split(X, y, test_size=0.3, random_state=42)


In [20]:
def systematic_sampling(X, y, step=2):
    idx = np.arange(0, len(X), step)
    return X[idx], y.iloc[idx]


In [21]:
def stratified_sampling(X, y):
    return train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


In [22]:
def cluster_sampling(X, y, clusters=5):
    df_temp = pd.DataFrame(X)
    df_temp['Class'] = y.values
    df_temp['cluster'] = np.random.randint(0, clusters, size=len(df_temp))

    for c in range(clusters):
        cluster_data = df_temp[df_temp['cluster'] == c]
        if cluster_data['Class'].nunique() > 1:
            return (
                cluster_data.drop(['Class', 'cluster'], axis=1).values,
                cluster_data['Class']
            )


In [23]:
def bootstrap_sampling(X, y):
    idx = np.random.choice(len(X), size=len(X), replace=True)
    return X[idx], y.iloc[idx]


In [26]:
models = {
    "M1_LogisticRegression": LogisticRegression(max_iter=1000),
    "M2_DecisionTree": DecisionTreeClassifier(),
    "M3_RandomForest": RandomForestClassifier(),
    "M4_SVM": SVC(),
    "M5_KNN": "knn"
}


In [28]:
results = {}

samplings = {
    "Sampling1_SimpleRandom": simple_random_sampling,
    "Sampling2_Systematic": systematic_sampling,
    "Sampling3_Stratified": stratified_sampling,
    "Sampling4_Cluster": cluster_sampling,
    "Sampling5_Bootstrap": bootstrap_sampling
}

for s_name, sampler in samplings.items():
    results[s_name] = {}

    if s_name in ["Sampling1_SimpleRandom", "Sampling3_Stratified"]:
        X_train, X_test, y_train, y_test = sampler(X, y)
    else:
        X_sample, y_sample = sampler(X, y)
        X_train, X_test, y_train, y_test = train_test_split(
            X_sample, y_sample, test_size=0.3, random_state=42
        )

    for m_name, model in models.items():

    # Safety: single-class data
        if len(np.unique(y_train)) < 2:
            results[s_name][m_name] = np.nan
            continue

        # 🔧 Dynamic KNN fix
        if model == "knn":
            k = min(5, len(X_train))
            if k < 2:
                results[s_name][m_name] = np.nan
                continue
            model = KNeighborsClassifier(n_neighbors=k)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[s_name][m_name] = round(accuracy_score(y_test, y_pred) * 100, 2)


In [29]:
results_df = pd.DataFrame(results)
results_df


,Sampling1_SimpleRandom,Sampling2_Systematic,Sampling3_Stratified,Sampling4_Cluster,Sampling5_Bootstrap
M1_LogisticRegression,33.33,0.00,16.67,50.0,100.00
M2_DecisionTree,66.67,0.00,66.67,0.0,100.00
M3_RandomForest,66.67,33.33,33.33,100.0,83.33
M4_SVM,50.00,33.33,16.67,50.0,100.00
M5_KNN,50.00,33.33,16.67,50.0,100.00


In [30]:
cv = StratifiedKFold(n_splits=5)
cv_model = RandomForestClassifier()

cv_scores = cross_val_score(cv_model, X, y, cv=cv)
cv_scores.mean() * 100


np.float64(60.0)